In [1]:
#Importando bibliotecas

import numpy as np

import torch
import torch.nn as nn
    
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
import pytorch_lightning as pl
from torch.utils.data import DataLoader

from pathlib import Path
from typing import List, Tuple

import numpy as np
import pandas as pd
import torch
import pytorch_lightning as pl
from torch.utils.data import DataLoader
import torch.nn as nn
import plotly.graph_objects as go

# CPC SSL Training

In [2]:
# Dados de Treino

data_path = Path('/workspaces/betania/ssl_tools/view_concatenated/train')

datas_x_train = []

data_y_train = []

for f in data_path.glob('*.csv'):
    data = pd.read_csv(f)
    x = data[['accel-x', 'accel-y', 'accel-z', 'gyro-x', 'gyro-y', 'gyro-z']].values

    # Expend dimension

    x = np.swapaxes(x, 1, 0)
    
    datas_x_train.append(x)

    y = data['standard activity code'].values

    data_y_train.append(y)



In [3]:
# Dados de Teste

data_path = Path('/workspaces/betania/ssl_tools/view_concatenated/test')

datas_x_test = []

data_y_test = []

for f in data_path.glob('*.csv'):
    data = pd.read_csv(f)
    x = data[['accel-x', 'accel-y', 'accel-z', 'gyro-x', 'gyro-y', 'gyro-z']].values

    # Expend dimension

    x = np.swapaxes(x, 1, 0)
    
    datas_x_test.append(x)

    y = data['standard activity code'].values

    data_y_test.append(y)

In [4]:
x_train = datas_x_train
y_train = data_y_train
x_test = datas_x_test
y_test = data_y_test


In [5]:
# Resume todos os vetores z em em único vetor de contexto​

class GRUEncoder(torch.nn.Module):
    def __init__(
        self,
        hidden_size: int = 100,
        in_channel: int = 6,
        encoding_size: int = 10,
        num_layers: int = 1,
        dropout: float = 0.0,
        bidirectional: bool = True,
        device: str = "cpu",
    ):
        super().__init__()
        self.hidden_size = hidden_size
        self.in_channel = in_channel
        self.num_layers = num_layers
        self.encoding_size = encoding_size
        self.bidirectional = bidirectional
        self.device = device
        
        self.rnn = torch.nn.GRU(
            input_size=self.in_channel,
            hidden_size=self.hidden_size,
            num_layers=num_layers,
            batch_first=False,
            dropout=dropout,
            bidirectional=bidirectional,
        ).to(device) 

        self.nn = torch.nn.Linear(
            self.hidden_size * (int(self.bidirectional) + 1), self.encoding_size
        ).to(device)

    def forward(self, x):
        x = x.permute(2, 0, 1)

        past = torch.zeros(
            self.num_layers * (int(self.bidirectional) + 1),
            x.shape[1],
            self.hidden_size,
            device=self.device,
        )

        out, _ = self.rnn(
            x, past
        )  # out shape = [seq_len, batch_size, num_directions*hidden_size]
        encodings = self.nn(out[-1].squeeze(0))
        return encodings

In [6]:
class CPC(pl.LightningModule):
    def __init__(
        self,
        encoder: torch.nn.Module,
        density_estimator: torch.nn.Module,
        auto_regressor: torch.nn.Module,
        lr: float = 1e-3,
        weight_decay: float = 0.0,
        window_size: int = 4,
        n_size: int = 5
    ):
        super().__init__()
        self.encoder = encoder.to(self.device)
        self.density_estimator = density_estimator.to(self.device)
        self.auto_regressor = auto_regressor.to(self.device)
        self.learning_rate = lr
        self.weight_decay = weight_decay
        self.window_size = window_size
        self.n_size = n_size
        self.training_step_losses = []


    def training_step(self, batch, batch_idx):
        assert len(batch) == 1, "Batch must be 1 sample only"
        sample = batch
        sample = sample.squeeze(0)
        rnd_t = np.random.randint(
            5 * self.window_size, sample.shape[-1] - 5 * self.window_size
        )
        sample = torch.tensor(
            sample[
                :,
                max(0, (rnd_t - 20 * self.window_size)) : min(
                    sample.shape[-1], rnd_t + 20 * self.window_size
                ),
            ]
        ).cpu()

        T = sample.shape[-1]
        windowed_sample = np.split(
            sample[:, : (T // self.window_size) * self.window_size],
            (T // self.window_size),
            -1,
        )
        windowed_sample = torch.tensor(np.stack(windowed_sample, 0), device=self.device)
        encodings = self.encoder(windowed_sample)
        window_ind = torch.randint(2, len(encodings) - 2, size=(1,))
        _, c_t = self.auto_regressor(
            encodings[max(0, window_ind[0] - 10) : window_ind[0] + 1].unsqueeze(0)
        )
        density_ratios = torch.bmm(
            encodings.unsqueeze(1),
            self.density_estimator(c_t.squeeze(1).squeeze(0)).expand_as(encodings).unsqueeze(-1),
        ).view(
            -1,
        )
        r = set(range(0, window_ind[0] - 2))
        r.update(set(range(window_ind[0] + 3, len(encodings))))
        rnd_n = np.random.choice(list(r), self.n_size)
        X_N = torch.cat(
            [density_ratios[rnd_n], density_ratios[window_ind[0] + 1].unsqueeze(0)], 0
        )
        labels = torch.Tensor([len(X_N) - 1]).to(self.device)
        loss = torch.nn.CrossEntropyLoss()(X_N.view(1, -1), labels.long())
        self.training_step_losses.append(loss)
        return loss


    def on_train_epoch_end(self) -> None:
        # do something with all training_step outputs, for example:
        epoch_mean = torch.stack(self.training_step_losses).mean()
        self.log("train_loss", epoch_mean, on_epoch=True, on_step=False, prog_bar=True, logger=True)
        # free up the memory
        self.training_step_losses.clear()
        
    def configure_optimizers(self):
        learnable_parameters = (
            list(self.density_estimator.parameters())
            + list(self.encoder.parameters())
            + list(self.auto_regressor.parameters())
        )

        optimizer = torch.optim.Adam(
            learnable_parameters, lr=self.learning_rate, weight_decay=self.weight_decay
        )
        return optimizer

In [7]:
encoding_size = 150
encoder = GRUEncoder(in_channel=6, encoding_size=150, device='cuda')
density_estimator = torch.nn.Linear(encoding_size, encoding_size)
auto_regressor = torch.nn.GRU(
    input_size=encoding_size, 
    hidden_size=encoding_size, 
    batch_first=True
)

cpc = CPC(encoder, density_estimator, auto_regressor)

In [8]:
class SimpleDataset:
    def __init__(self, X, y=None):
        self.X = X
        self.y = y
        
    def __len__(self):
        return len(self.X)
    
    def __getitem__(self, idx):
        if self.y is not None:
            return self.X[idx].astype(np.float32), self.y[idx].astype(np.float32)
        else:
            return self.X[idx].astype(np.float32)
    
train_dataset = SimpleDataset(x_train)
test_dataset = SimpleDataset(x_test, y_test)
len(train_dataset), len(test_dataset)

(21, 6)

In [9]:
train_loader = DataLoader(train_dataset, batch_size=1, num_workers=10, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=1, num_workers=10, shuffle=False)

In [10]:
trainer = pl.Trainer(max_epochs=500, accelerator="gpu", devices=1)

GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
/home/vscode/.local/lib/python3.10/site-packages/pytorch_lightning/trainer/connectors/logger_connector/logger_connector.py:67: Starting from v1.9.0, `tensorboardX` has been removed as a dependency of the `pytorch_lightning` package, due to potential conflicts with other packages in the ML ecosystem. For this reason, `logger=True` will use `CSVLogger` as the default logger, unless the `tensorboard` or `tensorboardX` packages are found. Please `pip install lightning[extra]` or one of them to enable TensorBoard support by default


In [11]:
trainer.fit(cpc, train_loader)

You are using a CUDA device ('NVIDIA A100 80GB PCIe') that has Tensor Cores. To properly utilize them, you should set `torch.set_float32_matmul_precision('medium' | 'high')` which will trade-off precision for performance. For more details, read https://pytorch.org/docs/stable/generated/torch.set_float32_matmul_precision.html#torch.set_float32_matmul_precision
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0,1,2,3,4,5,6,7]

  | Name              | Type       | Params
-------------------------------------------------
0 | encoder           | GRUEncoder | 95.0 K
1 | density_estimator | Linear     | 22.7 K
2 | auto_regressor    | GRU        | 135 K 
-------------------------------------------------
253 K     Trainable params
0         Non-trainable params
253 K     Total params
1.014     Total estimated model params size (MB)
/home/vscode/.local/lib/python3.10/site-packages/pytorch_lightning/loops/fit_loop.py:293: The number of training batches (21) is smaller than the logging interval Trainer(log_

Epoch 0:   0%|          | 0/21 [00:00<?, ?it/s] 

/tmp/ipykernel_314338/3311096431.py:30: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  sample = torch.tensor(


Epoch 499: 100%|██████████| 21/21 [00:00<00:00, 36.18it/s, v_num=138, train_loss=1.120]

`Trainer.fit` stopped: `max_epochs=500` reached.


Epoch 499: 100%|██████████| 21/21 [00:01<00:00, 10.53it/s, v_num=138, train_loss=1.120]


## CPC Fine-Tuning

We are going to fine-tune the CPC model on the downstream task of classification. We will use the same dataset and re-use the same encoder.

In [12]:
from typing import Any, Optional

from torchmetrics.functional import accuracy

# Classificando com uma MLP

class StateClassifier(torch.nn.Module):
    def __init__(self, input_size: int= 10, hidden_size1= 64, hidden_size2=64, num_classes= 6, dropout_prob= 0):
        super(StateClassifier, self).__init__()
        self.layer1 = nn.Sequential(
            nn.Linear(input_size, hidden_size1),
            nn.ReLU()
        )
        self.layer2 = nn.Sequential(
            nn.Linear(hidden_size1, hidden_size2),
            nn.ReLU()
        )
        self.dropout = nn.Dropout(p=dropout_prob)
        self.output_layer = nn.Sequential(
            nn.Linear(hidden_size2, num_classes),
            nn.Softmax(dim=1)
        )
        
    def forward(self, x):
        out = self.layer1(x)
        out = self.layer2(out)
        out = self.dropout(out)
        out = self.output_layer(out)
        return out

class CPC_Classifier(pl.LightningModule):
    def __init__(
        self,
        encoder: torch.nn.Module,
        classifier: torch.nn.Module,
        lr: float = 1e-3,
        weight_decay: float = 0.0,
        task_class: str = "multiclass",
        num_classes: int = 6
    ):
        super().__init__()
        self.encoder = encoder.to(self.device)
        self.classifier = classifier.to(self.device)
        self.learning_rate = lr
        self.weight_decay = weight_decay
        self.training_step_losses = []
        self.validation_step_losses = []
        self.loss_function = torch.nn.CrossEntropyLoss()
        self.task_class = task_class
        self.num_classes = num_classes
        
    def configure_optimizers(self) -> Any:
        optimizer = torch.optim.Adam(
            self.classifier.parameters(), lr=self.learning_rate, weight_decay=self.weight_decay
        )
        return optimizer
    
    def forward(self, x):
        encodings = self.encoder(x)
        predictions = self.classifier(encodings)
        return predictions
    
    def training_step(self, batch, batch_idx):
        x, y = batch
        predictions = self.forward(x)
        loss = self.loss_function(predictions, y.long())
        self.training_step_losses.append(loss)
        return loss
    
    def on_train_epoch_end(self) -> None:
        # do something with all training_step outputs, for example:
        epoch_mean = torch.stack(self.training_step_losses).mean()
        self.log("train_loss", epoch_mean, on_epoch=True, on_step=False, prog_bar=True, logger=True)
        # free up the memory
        self.training_step_losses.clear()
    
    def validation_step(self, batch, batch_idx):
        loss, acc = self._shared_eval_step(batch, batch_idx)
        self.validation_step_losses.append(loss)
        metrics = {"val_acc": acc, "val_loss": loss}
        self.log_dict(metrics)
        return metrics
    
    def test_step(self, batch, batch_idx):
        loss, acc = self._shared_eval_step(batch, batch_idx)
        metrics = {"test_acc": acc, "test_loss": loss}
        self.log_dict(metrics)
        return metrics

        
    def on_validation_epoch_end(self) -> None:
        # do something with all training_step outputs, for example:
        epoch_mean = torch.stack(self.validation_step_losses).mean()
        self.log("val_loss", epoch_mean, on_epoch=True, on_step=False, prog_bar=True, logger=True)
        # free up the memory
        self.validation_step_losses.clear()

    def _shared_eval_step(self, batch, batch_idx):
        x, y = batch
        predictions = self.forward(x)
        loss = self.loss_function(predictions, y.long())
        acc = accuracy(torch.argmax(predictions, dim=1), y.long(), task=self.task_class, num_classes=self.num_classes)
        return loss, acc

In [13]:
import pandas as pd
import numpy as np
import torch
from pathlib import Path
from torch.utils.data import TensorDataset, DataLoader

BATCH_SIZE = 100

# Caminho dos dados

data_path_train = Path('/workspaces/betania/ssl_tools/UCI/train.csv')

data_path_validation = Path('/workspaces/betania/ssl_tools/UCI/validation.csv')

data_path_test = Path('/workspaces/betania/ssl_tools/UCI/test.csv')

# Train

x_train = pd.read_csv(data_path_train)

x_train = x_train.iloc[:, :360]
    
x_train = x_train.astype(np.float32)

y_train = pd.read_csv(data_path_train)

y_train = y_train.iloc[:, -1]

y_train = y_train.astype(np.float32)

tensor_x = torch.Tensor(np.array(x_train))

tensor_y = torch.Tensor(np.array(y_train))

original_dim = tensor_x.shape[0]

input_shape = (original_dim, 6, 60)

tensor_x = tensor_x.reshape(input_shape)

dataset_train= TensorDataset(tensor_x,tensor_y)

dataloader_train = DataLoader(dataset_train, batch_size=BATCH_SIZE, shuffle=True, drop_last= True)

# Validation

x_validation = pd.read_csv(data_path_validation)

x_validation = x_validation.iloc[:, :360]

x_validation = x_validation.astype(np.float32)

y_validation = pd.read_csv(data_path_validation)

y_validation = y_validation.iloc[:, -1]

y_validation = y_validation.astype(np.float32)

tensor_x_val = torch.Tensor(np.array(x_validation))

tensor_y_val = torch.Tensor(np.array(y_validation))

original_dim_val = tensor_x_val.shape[0]

tensor_x_val = tensor_x_val.reshape(original_dim_val, 6, 60)

dataset_val= TensorDataset(tensor_x_val,tensor_y_val)

dataloader_val = DataLoader(dataset_val, batch_size=BATCH_SIZE, shuffle=True, drop_last = True)

# Test

x_test = pd.read_csv(data_path_test)

x_test = x_test.iloc[:, :360]

x_test = x_test.astype(np.float32)

y_test = pd.read_csv(data_path_test)

y_test = y_test.iloc[:, -1]

y_test = y_test.astype(np.float32)

tensor_x_test = torch.Tensor(np.array(x_test))

tensor_y_test = torch.Tensor(np.array(y_test))

original_dim_test = tensor_x_test.shape[0]

tensor_x_test = tensor_x_test.reshape((original_dim_test, 6, 60))

dataset_test= TensorDataset(tensor_x_test,tensor_y_test)

dataloader_test = DataLoader(dataset_test, batch_size=BATCH_SIZE, shuffle=True, drop_last= True)


In [14]:
data_path_train = Path('/workspaces/betania/ssl_tools/UCI/train.csv')

y_train = pd.read_csv(data_path_train)

y_train = y_train.iloc[:, -1]

y_train

0       2
1       2
2       2
3       2
4       2
       ..
2415    1
2416    1
2417    1
2418    1
2419    1
Name: standard activity code, Length: 2420, dtype: int64

In [15]:
# ver o numeros que estão na coluna de y

y_train.unique()


array([2, 3, 4, 0, 1])

In [16]:
# shape dataloader

for i, (x, y) in enumerate(dataloader_train):
    print(f"Batch {i} shape: {x.shape}")

for i, (x, y) in enumerate(dataloader_val):
    print(f"Batch {i} shape: {y.shape}")
    

Batch 0 shape: torch.Size([100, 6, 60])
Batch 1 shape: torch.Size([100, 6, 60])
Batch 2 shape: torch.Size([100, 6, 60])
Batch 3 shape: torch.Size([100, 6, 60])
Batch 4 shape: torch.Size([100, 6, 60])
Batch 5 shape: torch.Size([100, 6, 60])
Batch 6 shape: torch.Size([100, 6, 60])
Batch 7 shape: torch.Size([100, 6, 60])
Batch 8 shape: torch.Size([100, 6, 60])
Batch 9 shape: torch.Size([100, 6, 60])
Batch 10 shape: torch.Size([100, 6, 60])
Batch 11 shape: torch.Size([100, 6, 60])
Batch 12 shape: torch.Size([100, 6, 60])
Batch 13 shape: torch.Size([100, 6, 60])
Batch 14 shape: torch.Size([100, 6, 60])
Batch 15 shape: torch.Size([100, 6, 60])
Batch 16 shape: torch.Size([100, 6, 60])
Batch 17 shape: torch.Size([100, 6, 60])
Batch 18 shape: torch.Size([100, 6, 60])
Batch 19 shape: torch.Size([100, 6, 60])
Batch 20 shape: torch.Size([100, 6, 60])
Batch 21 shape: torch.Size([100, 6, 60])
Batch 22 shape: torch.Size([100, 6, 60])
Batch 23 shape: torch.Size([100, 6, 60])
Batch 0 shape: torch.Size(

In [17]:
encoding_size = 150
n_classes = 6

classifier = StateClassifier(input_size=encoding_size, num_classes=n_classes, dropout_prob=0, hidden_size1= 64, hidden_size2=32)
cpc_classifier = CPC_Classifier(encoder, classifier)

In [18]:
trainer = pl.Trainer(max_epochs=1000, accelerator="gpu", devices=1)

GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs


In [19]:
trainer.fit(cpc_classifier, train_dataloaders=dataloader_train, val_dataloaders=dataloader_val)

LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0,1,2,3,4,5,6,7]

  | Name          | Type             | Params
---------------------------------------------------
0 | encoder       | GRUEncoder       | 95.0 K
1 | classifier    | StateClassifier  | 11.9 K
2 | loss_function | CrossEntropyLoss | 0     
---------------------------------------------------
106 K     Trainable params
0         Non-trainable params
106 K     Total params
0.428     Total estimated model params size (MB)


/home/vscode/.local/lib/python3.10/site-packages/pytorch_lightning/trainer/connectors/data_connector.py:492: Your `val_dataloader`'s sampler has shuffling enabled, it is strongly recommended that you turn shuffling off for val/test dataloaders.
/home/vscode/.local/lib/python3.10/site-packages/pytorch_lightning/trainer/connectors/data_connector.py:441: The 'val_dataloader' does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` to `num_workers=127` in the `DataLoader` to improve performance.
/home/vscode/.local/lib/python3.10/site-packages/pytorch_lightning/trainer/connectors/data_connector.py:441: The 'train_dataloader' does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` to `num_workers=127` in the `DataLoader` to improve performance.
/home/vscode/.local/lib/python3.10/site-packages/pytorch_lightning/loops/fit_loop.py:293: The number of training batches (24) is smal

Epoch 1:  96%|█████████▌| 23/24 [00:00<00:00, 89.82it/s, v_num=139, val_loss=1.770, train_loss=1.780] 

Epoch 999: 100%|██████████| 24/24 [00:00<00:00, 112.90it/s, v_num=139, val_loss=1.230, train_loss=1.150]

`Trainer.fit` stopped: `max_epochs=1000` reached.


Epoch 999: 100%|██████████| 24/24 [00:00<00:00, 100.29it/s, v_num=139, val_loss=1.230, train_loss=1.150]


## Classification evaluation

In [20]:
trainer.test(cpc_classifier, dataloader_test)

LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0,1,2,3,4,5,6,7]
/home/vscode/.local/lib/python3.10/site-packages/pytorch_lightning/trainer/connectors/data_connector.py:492: Your `test_dataloader`'s sampler has shuffling enabled, it is strongly recommended that you turn shuffling off for val/test dataloaders.
/home/vscode/.local/lib/python3.10/site-packages/pytorch_lightning/trainer/connectors/data_connector.py:441: The 'test_dataloader' does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` to `num_workers=127` in the `DataLoader` to improve performance.


Testing DataLoader 0: 100%|██████████| 6/6 [00:00<00:00, 110.07it/s]
────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
       Test metric             DataLoader 0
────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
        test_acc            0.7983333468437195
        test_loss           1.2363642454147339
────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────


[{'test_acc': 0.7983333468437195, 'test_loss': 1.2363642454147339}]